<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/ReviewerResponseTable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 135 (delta 81), reused 95 (delta 41), pack-reused 0
Receiving objects: 100% (135/135), 61.46 KiB | 4.10 MiB/s, done.
Resolving deltas: 100% (81/81), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-pjq4sot3/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 1169, done.
remote: Counting objects: 100% (1169/1169), done.
remote: Compressing objects: 100% (1163/1163), done.
remote: Total 4107 (delta 72), reused 1048 (delta 5), pack-reused 2938
Receiving objects: 100% (4107/4107), 289.89 MiB | 48.45 MiB/s, done.
Resolving deltas: 100% (88/88), done.
Checking out files: 100% (3895/3895), done.


In [0]:
import gzip
import json

task0_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/hema/heme_task0_abstention_results.json.gz").read())
task1_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/hema/heme_task1_abstention_results.json.gz").read())
catdog_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/catdog/catdog_abstention_results.json.gz").read())

In [3]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np
from collections import defaultdict

datasetname_to_dataset = {
    'catdog': (catdog_abstention_results['metricname_to_abstfrac_to_abstnadapt_to_perf'],
               catdog_abstention_results['metricname_to_adaptname_to_baselineperfs']),
    'hemetask0': (task0_abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf'],
                  task0_abstention_results['metricname_to_baselineperfs']),
    'hemetask1': (task1_abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf'],
                  task1_abstention_results['metricname_to_baselineperfs'])
}

nicename_orders = [
    "$\Delta$Metric (Ours)",
    #"Est $\Delta$Metric (Ours)",
    "JS Div. (Ours)",
    #"JS Div. (Ours)",
    "Dist. from 0.5",
    "Dist. from 0.5",
    "Fumera et al.",
    "Fumera et al.",
    "Test-time Drop.",
    "Test-time Drop."
]

yesnoyesno = ['Y','Y','N','Y','N','Y','N','Y','N']

def get_method_orders(pc, metric, adapt):
  if adapt:
    possuffix='EM'
    negsuffix='None'
  else:
    possuffix='platt'
    negsuffix='none'   
  return [
      'mcwindowdelta_'+metric+'_'+pc+'pc:'+possuffix if metric=="tpratfpr" else 'estwindowdelta_'+metric+'_'+pc+'pc:'+possuffix,
      #'mcwindowdelta_'+metric+'_'+pc+'pc:'+negsuffix,
      'js:'+possuffix,
      #'js:'+negsuffix,
      'distfrom0.5:'+possuffix,
      'distfrom0.5:'+negsuffix,
      'fumera_'+metric+'_'+pc+'pc:'+possuffix,
      'fumera_'+metric+'_'+pc+'pc:'+negsuffix,
      'uncertainty:'+possuffix,
      'uncertainty:'+negsuffix, 
  ]
  

catdogtpr_abstfrac_to_methodorders = {
    '0.3': get_method_orders('30', 'tpratfpr', adapt=True),
    '0.15': get_method_orders('15', 'tpratfpr', adapt=True)
}
hemeauprc_abstfrac_to_methodorders = {
    '0.3': get_method_orders('30', 'auprc', adapt=False),
    '0.15': get_method_orders('15', 'auprc', adapt=False)
}

dataset_and_metricname_combos = [
    ('hemetask1', 'auprc', hemeauprc_abstfrac_to_methodorders),
    ('hemetask0', 'auprc', hemeauprc_abstfrac_to_methodorders),
    ('catdog', 'tpratfpr_0.01', catdogtpr_abstfrac_to_methodorders),
]

abstfracs = ['0.15', '0.3']


def stdderr(vals):
  return np.std(vals, ddof=1)/np.sqrt(len(vals))

def convert_method_to_perfs_to_ranks(method_to_perfs, method_names):
  n_samples = len(method_to_perfs[method_names[0]])
  method_to_ranks = defaultdict(list)
  for sampleidx in range(n_samples):
    method_and_perf_tuples = [
        (method_name, method_to_perfs[method_name][sampleidx])
         for method_name in method_names]
    sorted_method_and_perfs_tuples = sorted(method_and_perf_tuples,
                                            key=lambda x: -x[1])
    for i,(methodname,perf) in enumerate(sorted_method_and_perfs_tuples):
      method_to_ranks[methodname].append(i+1)
  return method_to_ranks
  
  

datasetnmetricname_to_abstfrac_to_bestmethods = defaultdict(dict)
datasetnmetricname_to_abstfrac_to_methodranks = defaultdict(dict)
for (datasetname, metricname, abstfrac_to_methodorders) in dataset_and_metricname_combos:  
  metricname_to_abstfrac_to_method_to_perf, metricname_to_baselineperfs =\
    datasetname_to_dataset[datasetname]
  #identify the best methods
  for abstfrac in abstfracs:
    method_names = abstfrac_to_methodorders[abstfrac]
    nsamples = len(list(metricname_to_abstfrac_to_method_to_perf
                        [metricname][abstfrac][method_names[0]]))
    method_to_perfs = metricname_to_abstfrac_to_method_to_perf[metricname][abstfrac]
    method_to_ranks = convert_method_to_perfs_to_ranks(
                        method_to_perfs=method_to_perfs,
                        method_names=method_names)
    datasetnmetricname_to_abstfrac_to_methodranks[
        datasetname+":"+metricname][abstfrac] = method_to_ranks
    ustats_mat = get_ustats_mat(
        method_to_perfs=method_to_perfs,
        method_names=method_names,
        max_ustat=nsamples*(nsamples+1)/2 )
    tied_top_methods, tied_worst_methods =(
        get_tied_top_and_worst_methods(
            ustats_mat,
            method_names,
            #0.05 threshold for one-sided test from normal approx when N=30,
            #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
            threshold=151
        ))
    tied_top_methods = [method_names[x] for x in tied_top_methods]
    print("bestmethods",metricname, abstfrac, tied_top_methods)
    datasetnmetricname_to_abstfrac_to_bestmethods[
        datasetname+":"+metricname][abstfrac] = set(tied_top_methods)
    


#generate the latex table
thetable = ("""
\\begin{table*}[!h]
\\tiny
\\begin{center}
\\begin{tabular}{ | c c | c c | c c | c c | }
\\hline
\\multicolumn{1}{| c }{ } & A/ & \\multicolumn{2}{ c |}{AuPRC (Leukemia Stem Cells) (0.695 $\\pm$ 0.002)}"""
                       +"""& \\multicolumn{2}{ c |}{AuPRC (pre-leukemic HSCs) (0.734 $\\pm$ 0.002) }"""
                       +"""& \\multicolumn{2}{ c |}{Cat v Dog Sens. @ 99\\% Spec. (0.332 $\\pm$ 0.006)}\\\\
\\multicolumn{1}{| c}{Method} & \\multicolumn{1}{c |}{C?} & @30\\% Abst; avg. rank & @15\\% Abst; avg. rank & @30\\% Abst; avg. rank & @15\\% Abst; avg. rank & @30\\% Abst; avg. rank & @15\\% Abst; avg. rank\\\\ \\hline
""")
for rowidx in range(len(nicename_orders)):
  nicemethodname = nicename_orders[rowidx]
  yesorno = yesnoyesno[rowidx]
  thetable += nicemethodname
  thetable += " & "+yesorno
  for (datasetname, metricname, abstfrac_to_methodorders) in dataset_and_metricname_combos: 
    (metricname_to_abstfrac_to_methodname_to_perf,
     metricname_to_baselineperfs)  = datasetname_to_dataset[datasetname]
    #index by adapt-or-not, if needed
    if (isinstance(metricname_to_baselineperfs[metricname], list)):
      baseperfs = metricname_to_baselineperfs[metricname]
    else:
      baseperfs = metricname_to_baselineperfs[metricname][
                   abstfrac_to_methodorders['0.3'][rowidx].split(":")[1]]    
    print(np.round(np.mean(baseperfs),4), np.round(stdderr(baseperfs),3))
    #thetable += (" & "+str(np.round(np.mean(baseperfs),4))
                 #+"$\\pm$"+str(np.round(stdderr(baseperfs),4))
    #            )
    for abstfrac in ['0.3', '0.15']:
      methodname = abstfrac_to_methodorders[abstfrac][rowidx]
      isbest = (methodname in
        datasetnmetricname_to_abstfrac_to_bestmethods[
            datasetname+":"+metricname][abstfrac])
      
      ranks = datasetnmetricname_to_abstfrac_to_methodranks[
          datasetname+":"+metricname][abstfrac][methodname]
      
      methodperfs = metricname_to_abstfrac_to_methodname_to_perf[
        metricname][abstfrac][methodname]
      thetable += " & "
      if (isbest):
        thetable += "\\textbf{"
      thetable += (str(np.round(np.mean(methodperfs),3))
                   +"$\pm$"+str(np.round(stdderr(methodperfs),3))
                   +"; "+str(np.round(np.mean(ranks),1))
                        +"$\pm$"+str(np.round(stdderr(ranks),1)))
      if (isbest):
        thetable += "}"
  thetable += "\\\\ \\hline\n"
  
thetable += """\\end{tabular}
\\end{center}
\\caption{\\textbf{}. }
\\label{tab:reviewers}
\\end{table*}
"""
print("\n\n"+thetable)

bestmethods auprc 0.15 ['estwindowdelta_auprc_15pc:platt']
bestmethods auprc 0.3 ['estwindowdelta_auprc_30pc:platt']
bestmethods auprc 0.15 ['estwindowdelta_auprc_15pc:platt']
bestmethods auprc 0.3 ['estwindowdelta_auprc_30pc:platt']
bestmethods tpratfpr_0.01 0.15 ['mcwindowdelta_tpratfpr_15pc:EM']
bestmethods tpratfpr_0.01 0.3 ['mcwindowdelta_tpratfpr_30pc:EM', 'fumera_tpratfpr_30pc:None']
0.6946 0.002
0.7341 0.002
0.3318 0.006
0.6946 0.002
0.7341 0.002
0.3318 0.006
0.6946 0.002
0.7341 0.002
0.3318 0.006
0.6946 0.002
0.7341 0.002
0.3318 0.006
0.6946 0.002
0.7341 0.002
0.3318 0.006
0.6946 0.002
0.7341 0.002
0.3318 0.006
0.6946 0.002
0.7341 0.002
0.3318 0.006
0.6946 0.002
0.7341 0.002
0.3318 0.006



\begin{table*}[!h]
\tiny
\begin{center}
\begin{tabular}{ | c c | c c | c c | c c | }
\hline
\multicolumn{1}{| c }{ } & A/ & \multicolumn{2}{ c |}{AuPRC (Leukemia Stem Cells) (0.695 $\pm$ 0.002)}& \multicolumn{2}{ c |}{AuPRC (pre-leukemic HSCs) (0.734 $\pm$ 0.002) }& \multicolumn{2}{ c |}{Ca